##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Clasificación de textos con TensorFlow Hub: reseñas de películas

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar notebook</a> </td>
  <td>     <a href="https://tfhub.dev/s?module-type=text-embedding"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelos de TF Hub</a> </td>
</table>

En este bloc de notas se clasifican reseñas de películas como *positiva* o *negativa* a partir del texto de la reseña. Este es un ejemplo de clasificación *binaria* (o de dos clases), un tipo de problema de aprendizaje automático importante y ampliamente aplicable.

En este tutorial se demuestra la aplicación básica del aprendizaje por transferencia con [TensorFlow Hub](https://tfhub.dev) y Keras.

Usa los [conjuntos de datos de IMDB](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) que contiene el texto de 50 000 reseñas de películas de [Internet Movie Database](https://www.imdb.com/). Se divide en 25 000 reseñas para entrenamiento y 25 000 reseñas para prueba. Los conjuntos de entrenamiento y prueba están *equilibrados*, lo que significa que contienen la misma cantidad de reseñas positivas y negativas.

En este bloc de notas se usa [`tf.keras`](https://www.tensorflow.org/guide/keras), una API de alto nivel que se usa para desarrollar y entrenar modelos en TensorFlow, y [`tensorflow_hub`](https://www.tensorflow.org/hub), una biblioteca que sirve para cargar modelos entrenados desde [TFHub](https://tfhub.dev) en una única línea de código. Si desea obtener un tutorial más avanzado sobre clasificación de textos usando `tf.keras`, consulte la [Guía de clasificación de textos de MLCC](https://developers.google.com/machine-learning/guides/text-classification/).

In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [ ]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

## Descargar el conjunto de datos de IMDB

El conjunto de datos de IMDB está disponible en [reseñas de imdb](https://www.tensorflow.org/datasets/catalog/imdb_reviews) o en [conjuntos de datos de TensorFlow](https://www.tensorflow.org/datasets). El siguiente código sirve para descargar el conjunto de datos de IMDB en su máquina (o en el tiempo de ejecución de colab):

In [ ]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

## Explorar los datos

Tomémonos un momento para comprender el formato de los datos. Cada ejemplo es una frase que representa la reseña de película y una etiqueta correspondiente. La frase no se preprocesa de ninguna manera. La etiqueta es un valor de número entero que puede ser 0 o 1, donde 0 corresponde a una reseña negativa y 1 corresponde a una reseña positiva.

Imprimamos los primeros 10 ejemplos.

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

Imprimamos también las primeras 10 etiquetas.

In [ ]:
train_labels_batch

## Generar el modelo

La red neuronal se crea apilando capas, lo que implica tomar tres decisiones principales en términos de arquitectura:

- ¿Cómo se representará el texto?
- ¿Cuántas capas se usarán en el modelo?
- ¿Cuántas *unidades ocultas* se usarán para cada capa?

En este ejemplo, los datos de entrada están compuestos por frases. Las etiquetas que se deben predecir son 0 o 1.

Una forma de representar el texto es convertir las frases en vectores incorporados. Usar una incorporación de texto previamente entrenada como la primera capa, lo que nos dará tres ventajas:

- No tendrá que preocuparse por el preprocesamiento del texto.
- Se beneficiará del aprendizaje por transferencia.
- La incorporación tiene un tamaño fijo, por lo que es más fácil de procesar.

Para este ejemplo usará un **modelo de incorporación de texto preentrenado** de [TensorFlow Hub](https://tfhub.dev) conocido como [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2).

Hay muchas otras incorporaciones de texto preentrenadas de TFHub que se pueden usar en este tutorial:

- [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) - que se entrenó con la misma arquitectura NNLM en los mismos datos que [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2), pero con una dimensión de incorporación más amplia. Las incorporaciones de mayor dimensión pueden mejorar la tarea, pero es posible que se necesite más tiempo para entrenar el modelo.
- [google/nnlm-en-dim128-with-normalization/2](https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2) - es igual a [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2), pero con una normalización adicional del texto, como la eliminación de puntuación. Esto puede ser útil si el texto de su tarea contiene puntuación o caracteres adicionales.
- [google/universal-sentence-encoder/4](https://tfhub.dev/google/universal-sentence-encoder/4) - un modelo mucho más grande que produce 512 incorporaciones dimensionales entrenadas con un codificador de red de promediado profundo (DAN).

¡Y muchas más! Busque [modelos de incorporación de texto](https://tfhub.dev/s?module-type=text-embedding) en TFHub.

En primer lugar, debemos crear una capa de Keras que use un modelo de TensorFlow Hub para incorporar las frases y probarlo con un par de ejemplos de entrada. Tenga en cuenta que, independientemente de la longitud del texto de entrada, la forma de la salida de las incorporaciones será de este modo: `(num_examples, embedding_dimension)`.

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

Ahora vamos a generar un modelo completo:

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Las capas se apilan secuencialmente para generar el clasificador:

1. La primera capa es una capa de TensorFlow Hub. Esta capa usa un modelo guardado preentrenado para asignar una frase a su vector de incorporación. El modelo de incorporación de texto preentrenado que está usando ([google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2)) separa la frase en tokens, incorpora cada token y luego combina la incorporación. De esto surgen las siguientes dimensiones: `(num_examples, embedding_dimension)`. Para este modelo NNLM, `embedding_dimension` equivale a 50.
2. Este vector de salida de longitud fija se canaliza a una capa (`Dense`) completamente conectada con 16 unidades ocultas.
3. La última capa está densamente conectada con un único nodo de salida.

Compilemos el modelo.

### Función de pérdida y optimizador

Un modelo necesita una función de pérdida y un optimizador para el entrenamiento. Dado que este es un problema de clasificación binaria y el modelo genera logits (una capa de una sola unidad con una activación lineal), usaremos la función de pérdida `binary_crossentropy`.

Esta no es la única opción para una función de pérdida, usted podría, por ejemplo, elegir `mean_squared_error`. Pero, por lo general, `binary_crossentropy` es mejor para trabajar con probabilidades; mide la "distancia" entre las distribuciones de probabilidad o, en nuestro caso, entre la distribución real y las predicciones.

Más adelante, cuando exploremos los problemas de regresión (por ejemplo, para predecir el precio de una vivienda), veremos cómo usar otra función de pérdida conocida como error cuadrático medio.

Ahora, configure el modelo para usar un optimizador y una función de pérdida:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Entrenar el modelo

Entrene el modelo durante 10 épocas en minilotes de 512 muestras. Es decir, 10 iteraciones sobre todas las muestras en los tensores `x_train` y `y_train`. Durante el entrenamiento, monitoree la pérdida y la precisión del modelo en las 10 000 muestras del conjunto de validación:

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

## Evaluar el modelo

Y veamos el rendimiento del modelo. Nos devolverá dos valores; la pérdida (un número que representa nuestro error, los valores bajos son mejores) y la precisión.

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

Este enfoque, relativamente sencillo, alcanza una precisión de aproximadamente un 87 %. Con enfoques más avanzados, el modelo debería acercarse al 95 %.

## Seguir leyendo

- Si desea conocer una forma más genérica de trabajar con entradas de cadenas y obtener un análisis más detallado del progreso de la precisión y la pérdida durante el entrenamiento, consulte el tutorial [Clasificación de textos con texto preprocesado](./text_classification.ipynb).
- Pruebe más [tutoriales relacionados con textos](https://www.tensorflow.org/hub/tutorials#text-related-tutorials) a partir del uso de modelos entrenados de TFHub.